<h1 style="background-color:rgba(203, 35, 23, 0.92); font-family: newtimeroman; font-size: 250%; color: white; border-radius: 80px 20px; padding: 25px; text-align: center;">
  <b>Obesity Risk Prediction</b>
</h1>

**Dataset Overview**

This notebook aims to analyze and predict obesity levels based on lifestyle, dietary habits, and personal characteristics.

**Key Features**  

- **id:** Unique identifier for each individual in the dataset.  
- **Gender:** Biological sex of the individual (Male/Female).  
- **Age:** Age of the person (years).  
- **Height:** Height of the individual (meters).  
- **Weight:** Weight of the individual (kilograms).  
- **family_history_with_overweight:** Indicates if obesity runs in the family (Yes/No).  
- **FAVC:** Frequent consumption of high-calorie food (Yes/No).  
- **FCVC:** Frequency of vegetable consumption (Scale: 1-3).  
- **NCP:** Number of main meals consumed daily.  
- **CAEC:** Frequency of food consumption between meals (Never/Sometimes/Frequently/Always).  
- **SMOKE:** Indicates if the person smokes (Yes/No).  
- **CH2O:** Daily water intake (liters).  
- **SCC:** Whether the person monitors their calorie intake (Yes/No).  
- **FAF:** Frequency of physical activity (Scale: 0-3).  
- **TUE:** Time spent using electronic devices daily (hours).  
- **CALC:** Alcohol consumption frequency (Never/Sometimes/Frequently/Always).  
- **MTRANS:** Mode of transportation (Public Transport/Walking/Automobile/Bike).  
- **NObeyesdad:** Target variable - obesity classification (Normal Weight, Overweight, Obese, etc.).  


In [3]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
train = pd.read_csv('../data/train.csv')    
test = pd.read_csv('../data/test.csv')  

In [13]:
train.shape

(20758, 18)